In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Display/Hide raw code."></form>''')

<center><h2> Optical simulation of 10nm SiO2 on Si</h2></center>
Here is an example of the transfert matrix function to simulate 10 nm SiO2 knowing the refractive index of the substrate Si. The dielectric SiO2 layer is simulated by a Lorentz oscillator model.

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from scipy.optimize import leastsq
import math
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import inspect
%matplotlib inline
#----Initiating wavelength and by reading data from file----#
my_data = np.genfromtxt('10nm _SiO2_on_Si.csv', delimiter=';')

def n(eps):#Function to convert complex permittivity to refractive index
    reps=np.real(eps)
    imeps=np.imag(eps)
    n=np.sqrt((np.sqrt(reps**2+imeps**2)+reps)/2)
    return n

def k(eps):#Function to convert complex permittivity to extinction coefficient
    reps=np.real(eps)
    imeps=np.imag(eps)
    k=np.sqrt((np.sqrt(reps**2+imeps**2)-reps)/2)
    return k

def Lorentz(F,gamma,E0,E):#Lorentz oscillator to model permittivity dispersion law of a dielectric
    eps=1+(F**2/(E0**2-E**2-1j*gamma*E))
    return eps

def TLorentz(epsinf,F,C,E0,Eg,E):##Lorentz oscillator to model permittivity dispersion law  of a dielectric close to bandgap
    ieps=(F*E0*C*(E-Eg)**2)/(((E**2-E0**2)**2+C**2*E**2)*E)*np.heaviside(E-Eg,0)
    alpha=np.sqrt(4*E0**2-C**2)
    gamma=np.sqrt(E0**2-C**2/2)
    aln=(Eg**2-E0**2)*E**2+(Eg**2*C**2)-E0**2*(E0**2+3*Eg**2)
    atan=(E**2-E0**2)*(E0**2+Eg**2)+Eg**2*C**2
    xsi4=(E**2-gamma**2)**2+alpha**2*C**2/4
    reps=epsinf+(F*C*aln/(np.pi*xsi4*2*alpha*E0))*np.log((E0**2+Eg**2+alpha*Eg)/(E0**2+Eg**2-alpha*Eg))-((F*atan/(np.pi*xsi4*E0))*(np.pi-np.arctan((2*Eg+alpha)/C)+np.arctan((-2*Eg+alpha)/C)))+((2*F*E0/(np.pi*xsi4*alpha)*Eg*(E**2-gamma**2)*(np.pi+(2*np.arctan(2*(gamma**2-Eg**2)/(alpha*C))))))-((F*E0*C*(E**2+Eg**2))/(np.pi*xsi4*E)*np.log(np.absolute(E-Eg)/(E+Eg)))+((2*F*E0*C*Eg)/(np.pi*xsi4)*np.log((np.absolute(E-Eg)*(E+Eg))/np.sqrt((E0**2+Eg**2)**2+Eg**2*C**2)))
    eps=reps+1j*ieps
    return eps

tlam=[]
tdel=[]
tpsi=[]
for i in range(len(my_data)):
    tlam.append(my_data[i][0]*1e-9)#convert input wavelength from nm to m
    tpsi.append(my_data[i][1])
    tdel.append(my_data[i][2])

#wavelength extremum are extracted from file, number of point is a parameter
lam_min=min(tlam) #minimum wavelength in m from data
lam_max=max(tlam)#maximum wavelength in m from data
nb=100 #number of frequency points length of data by default but can be change to arbitrary number
#interpolation of data points to have any number of points of wavelength wanted
interpoled_psi = interp1d(tlam, tpsi, kind='cubic')
interpoled_delta= interp1d(tlam, tdel, kind='cubic')
psi_e=interpoled_psi(np.linspace(lam_min,lam_max,nb))
delta_e=interpoled_delta(np.linspace(lam_min,lam_max,nb))
lam=np.linspace(lam_min,lam_max,nb)
c=299792458 #speed of light
omega=2*c*math.pi/lam #Setting angular frequency array
hb=6.582119514*10**-16#planck constant
E=omega*hb;#Setting energy array

tlam=[]
tn=[]
tk=[]
my_data = np.genfromtxt('Si.csv', delimiter=';')
for i in range(len(my_data)):
    tlam.append(my_data[i][0]*1e-9)#convert input wavelength from nm to m
    tn.append(my_data[i][1])
    tk.append(my_data[i][2])
interpoled_n = interp1d(tlam, tn, kind='linear')
interpoled_k= interp1d(tlam, tk, kind='linear')
nsub=interpoled_n(np.linspace(lam_min,lam_max,nb))
ksub=interpoled_k(np.linspace(lam_min,lam_max,nb))
n_i=np.linspace(1,1,nb)
Nsub=nsub-ksub*1j

def fit(d,F,gamma,E0):
    d=d*10**-9#convert input thickness from nm to m
    d2=1e-3 #thickness of the substrate
    
    #Euler angles of the optical layer
    phi_E=math.radians(0)
    theta_E=math.radians(0)
    psi_E=math.radians(0)
    
    theta_i=math.radians(70)#Incidence angle
    #----Definition of the Euler angles----#
    c1=np.cos(phi_E)
    c2=np.cos(theta_E)
    c3=np.cos(psi_E)
    s1=np.sin(phi_E)
    s2=np.sin(theta_E)
    s3=np.sin(psi_E)
    Euler=np.matrix([[c1*c3-(c2*s1*s3),-c1*s3-(c2*c3*s1),s1*s2],
            [c3*s1+(c1*c2*s3),c1*c2*c3-(s1*s3),-c1*s2],
            [s2*s3, c3*s2,c2]])

    nx=ny=nz=n(Lorentz(F,gamma,E0,E))
    kx=ky=kz=k(Lorentz(F,gamma,E0,E))
    
    epsx=nx**2-kx**2+1j*(2*nx*kx)
    epsy=ny**2-ky**2+1j*(2*ny*ky)
    epsz=nz**2-kz**2+1j*(2*nz*kz)

    def Ti():
        Kxx=n_i*np.sin(theta_i)
        epsp =[np.matrix([[epsx[i],0,0],[0, epsy[i],0],[0, 0,epsz[i]]]) for i in range(nb)]
        if(np.all(epsx==epsy)and np.all(epsx==epsz)): 
            # if layer is istoropic the eigen value can be express as :
            q=[np.sqrt(epsx[i]-Kxx[i]**2) for i in range(nb)]
            Ti=[np.matrix([[np.cos(omega[i]/c*d*q[i]),0,0,np.sin(omega[i]/c*d*q[i])*q[i]*1j/epsx[i]],
                        [0,np.cos(omega[i]/c*d*q[i]),np.sin(omega[i]/c*d*q[i])*(-1j)/q[i],0],
                        [0,np.sin(omega[i]/c*d*q[i])*q[i]*(-1j),np.cos(omega[i]/c*d*q[i]),0],
                        [np.sin(omega[i]/c*d*q[i])*1j*epsx[i]/q[i],0,0,np.cos(omega[i]/c*d*q[i])]]) for i in range(nb)]
            return Ti
        else: # if layer is anistoropic 
            #Computation of the dielectric tensor given the euler angle of the thin film
            eps=[np.dot(np.dot(Euler,epsp[i]),Euler.transpose()) for i in range(nb)]
        #Calculus of the delta matrix#
            delta=[np.matrix([[-Kxx[i]*(eps[i][2,0]/eps[i][2,2]),-Kxx[i]*(eps[i][2,1]/eps[i][2,2]),0,1-(Kxx[i]*Kxx[i]/eps[i][2,2])],
                    [0,0,-1,0],
                    [(eps[i][1,2]*eps[i][2,0]/eps[i][2,2])-eps[i][1,0],Kxx[i]*Kxx[i]-eps[i][1,1]+(eps[i][1,2]*eps[i][2,1]/eps[i][2,2]),0,(Kxx[i]*eps[i][1,2]/eps[i][2,2])],
                    [(-eps[i][0,2]*eps[i][2,0]/eps[i][2,2])+eps[i][0,0],-(eps[i][0,2]*eps[i][2,1]/eps[i][2,2])+eps[i][0,1],0,(-Kxx[i]*eps[i][0,2]/eps[i][2,2])]]) for i in range(nb)]
        #Calculus of the eigenvalues matrix#
            q=[np.linalg.eig(delta[i])[0] for i in range(nb)]
        
        #Calculus of the eigenvalues matrix#
            beta_0=[-q[i][1]*q[i][2]*q[i][3]*np.exp(-d*1j*q[i][0]*omega[i]/c)/((q[i][0]-q[i][1])*(q[i][0]-q[i][2])*(q[i][0]-q[i][3]))
            -q[i][2]*q[i][3]*q[i][0]*np.exp(-d*1j*q[i][1]*omega[i]/c)/((q[i][1]-q[i][2])*(q[i][1]-q[i][3])*(q[i][1]-q[i][0]))
            -q[i][3]*q[i][0]*q[i][1]*np.exp(-d*1j*q[i][2]*omega[i]/c)/((q[i][2]-q[i][3])*(q[i][2]-q[i][0])*(q[i][2]-q[i][1]))
            -q[i][0]*q[i][1]*q[i][2]*np.exp(-d*1j*q[i][3]*omega[i]/c)/((q[i][3]-q[i][0])*(q[i][3]-q[i][1])*(q[i][3]-q[i][2])) for i in range(nb)]

            beta_1=[((q[i][1]*q[i][2])+(q[i][1]*q[i][3])+(q[i][2]*q[i][3]))*np.exp(-d*1j*q[i][0]*omega[i]/c)/((q[i][0]-q[i][1])*(q[i][0]-q[i][2])*(q[i][0]-q[i][3]))
            +((q[i][2]*q[i][3])+(q[i][2]*q[i][0])+(q[i][3]*q[i][0]))*np.exp(-d*1j*q[i][1]*omega[i]/c)/((q[i][1]-q[i][2])*(q[i][1]-q[i][3])*(q[i][1]-q[i][0]))
            +((q[i][3]*q[i][0])+(q[i][3]*q[i][1])+(q[i][0]*q[i][1]))*np.exp(-d*1j*q[i][2]*omega[i]/c)/((q[i][2]-q[i][3])*(q[i][2]-q[i][0])*(q[i][2]-q[i][1]))
            +((q[i][0]*q[i][1])+(q[i][0]*q[i][2])+(q[i][1]*q[i][2]))*np.exp(-d*1j*q[i][3]*omega[i]/c)/((q[i][3]-q[i][0])*(q[i][3]-q[i][1])*(q[i][3]-q[i][2])) for i in range(nb)]

            beta_2=[-(q[i][1]+q[i][2]+q[i][3])*np.exp(-d*1j*q[i][0]*omega[i]/c)/((q[i][0]-q[i][1])*(q[i][0]-q[i][2])*(q[i][0]-q[i][3]))
            -(q[i][2]+q[i][3]+q[i][0])*np.exp(-d*1j*q[i][1]*omega[i]/c)/((q[i][1]-q[i][2])*(q[i][1]-q[i][3])*(q[i][1]-q[i][0]))
            -(q[i][3]+q[i][0]+q[i][1])*np.exp(-d*1j*q[i][2]*omega[i]/c)/((q[i][2]-q[i][3])*(q[i][2]-q[i][0])*(q[i][2]-q[i][1]))
            -(q[i][0]+q[i][1]+q[i][2])*np.exp(-d*1j*q[i][3]*omega[i]/c)/((q[i][3]-q[i][0])*(q[i][3]-q[i][1])*(q[i][3]-q[i][2])) for i in range(nb)]

            beta_3=[np.exp(-d*1j*q[i][0]*omega[i]/c)/((q[i][0]-q[i][1])*(q[i][0]-q[i][2])*(q[i][0]-q[i][3]))
            +np.exp(-d*1j*q[i][1]*omega[i]/c)/((q[i][1]-q[i][2])*(q[i][1]-q[i][3])*(q[i][1]-q[i][0]))
            +np.exp(-d*1j*q[i][2]*omega[i]/c)/((q[i][2]-q[i][3])*(q[i][2]-q[i][0])*(q[i][2]-q[i][1]))
            +np.exp(-d*1j*q[i][3]*omega[i]/c)/((q[i][3]-q[i][0])*(q[i][3]-q[i][1])*(q[i][3]-q[i][2])) for i in range(nb)]

            Ti=[beta_0[i]*np.identity(4)+beta_1[i]*delta[i]+beta_2[i]*np.dot(delta[i],delta[i])+beta_3[i]*np.dot(delta[i],np.dot(delta[i],delta[i])) for i in range(nb)]
        return Ti

    def Tp():
        nTi=Ti()
        nt=Nsub
        costheta_t=[np.sqrt(1-(n_i[i]*np.sin(theta_i)**2/(nt[i]**2))) for i in range(nb)]
        Lt=[np.matrix([[0,0,costheta_t[i],-costheta_t[i]],[1,1,0,0],[-nt[i]*costheta_t[i],nt[i]*costheta_t[i],0,0],[0,0,nt[i],nt[i]]]) for i in range(nb)]
        Li=[1/2*np.matrix([[0,1,-1/(n_i[i]*np.cos(theta_i)),0],[0,1,1/(n_i[i]*np.cos(theta_i)),0],[1/np.cos(theta_i),0,0,n_i[i]],[-1/np.cos(theta_i),0,0,n_i[i]]]) for i in range(nb)]
        Tp=[np.dot(Li[i],np.dot(nTi[i],Lt[i]))for i in range(nb)] 
        return Tp

    def Jr():
        T=Tp()
        #Calculus of the Jones reflectance coefficients at the interface#
        Rpp=[(T[i][0,0]*T[i][3,2]-T[i][0,2]*T[i][3,0])/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Rsp=[(T[i][0,0]*T[i][1,2]-T[i][0,2]*T[i][1,0])/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Rss=[(T[i][1,0]*T[i][2,2]-T[i][1,2]*T[i][2,0])/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Rps=[(T[i][2,2]*T[i][3,0]-T[i][2,0]*T[i][3,2])/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Jr=[[[Rpp[i],Rps[i]],[Rsp[i],Rss[i]]]for i in range(nb)]
        return Jr

    def Jt():
        T=Tp()
        #Calculus of the Jones tansmittance coefficients at the interface#
        Tss=[T[i][2,2]/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Tsp=[-T[i][2,0]/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Tps=[-T[i][0,2]/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Tpp=[T[i][0,0]/(T[i][0,0]*T[i][2,2]-T[i][0,2]*T[i][2,0]) for i in range(nb)]
        Jt=[[[Tpp[i],Tps[i]],[Tsp[i],Tss[i]]]for i in range(nb)]
        return Jt

    def Mjr():
        nJr=Jr()
        #Calculus of the Mueller matrices from Jones matrix#
        A=[[1,0,0,1],[-1,0,0,1],[0,1,1,0],[0,1j,-1j,0]]
        Mjr=[np.dot(A,np.dot(np.kron(nJr[i],np.conjugate(nJr[i])),np.linalg.inv(A)))for i in range(nb)]
        return Mjr

    def Mjt():
        #Calculus of the Mueller matrices from Jones matrix#
        nJt=Jt()
        A=[[1,0,0,1],[-1,0,0,1],[0,1,1,0],[0,1j,-1j,0]]
        Mjt=[np.dot(A,np.dot(np.kron(nJt[i],np.conjugate(nJt[i])),np.linalg.inv(A)))for i in range(nb)]
        return Mjt

    def Mnjr():
        nMjr=Mjr()
        Mnjr=[nMjr[i]/nMjr[i][0,0] for i in range(nb)]
        return Mnjr
        
    def Mnjt():
        nMjt=Mjt()
        Mnjt=[nMjt[i]/nMjt[i][0,0] for i in range(nb)]
        return Mnjt

    def Tft():
        nJt=Jt()
        ###Calcul of the substrate contribution###
        nt=Nsub
        costheta_t=[np.sqrt(1-(n_i[i]*np.sin(theta_i)**2/(nt[i]**2))) for i in range(nb)]
        
        theta_1=[np.arccos(costheta_t[i]) for i in range(nb)]
        theta_2=[np.arcsin(Nsub[i]*np.sin(theta_1[i])/n_i[i]) for i in range(nb)]

        Rbp=[(n_i[i]*np.cos(theta_1[i])-Nsub[i]*np.cos(theta_2[i]))/(Nsub[i]*np.cos(theta_1[i])+n_i[i]*np.cos(theta_2[i])) for i in range(nb)]
        Rbs=[(Nsub[i]*np.cos(theta_1[i])-n_i[i]*np.cos(theta_2[i]))/(Nsub[i]*np.cos(theta_1[i])+n_i[i]*np.cos(theta_2[i])) for i in range(nb)]
        Tbp=[(2*Nsub[i]*np.cos(theta_1[i]))/(Nsub[i]*np.cos(theta_2[i])+n_i[i]*np.cos(theta_1[i])) for i in range(nb)]
        Tbs=[(2*Nsub[i]*np.cos(theta_1[i]))/(Nsub[i]*np.cos(theta_1[i])+n_i[i]*np.cos(theta_2[i])) for i in range(nb)]

        Jf=[np.matrix([[Tbp[i],0],[0,Tbs[i]]]) for i in range(nb)]
        k0=omega/c
        eps=[np.matrix([[Nsub[i]**2,0,0],[0, Nsub[i]**2,0],[0, 0,Nsub[i]**2]]) for i in range(nb)]
        Kxx=[n_i[i]*np.sin(theta_i) for i in range(nb)]
        q=[np.sqrt(eps[i][0,0]-Kxx[i]**2) for i in range(nb)]
        Jtsub=[np.matrix([[np.exp(1j*k0[i]*d2*q[i]),0],[0,np.exp(1j*k0[i]*d2*q[i])]]) for i in range(nb)]
        Tt=[np.dot(Jf[i],np.dot(Jtsub[i],nJt[i])) for i in range(nb)]
        Tft=[Tt[i][0,0]*np.conjugate(Tt[i][0,0]) for i in range(nb)]
        return Tft

    def psi():
        J=Jr()
        rp = [np.absolute(J[i][0][0]) for i in range(nb)]
        rs = [np.absolute(J[i][1][1]) for i in range(nb)]
        return np.arctan2(rp, rs)* 180 / np.pi

    def delta():
        J=Jr()
        rp= [J[i][0][0] for i in range(nb)]
        rs= [J[i][1][1] for i in range(nb)]
        return np.asarray([np.angle(rp[i]/rs[i], deg=True) for i in range(nb)])

    return {'psi': psi(), 'delta': delta()}



In [3]:
d_widget = widgets.FloatSlider(10,min=0.0, max=50, step=1)
F_widget = widgets.FloatSlider(18,min=0.0, max=50, step=1)
gamma_widget = widgets.FloatSlider(0.0,min=0.0, max=50, step=1)
E0_widget = widgets.FloatSlider(25,min=0.0, max=50, step=1)
#u = widgets.HBox([d_widget,F_widget,gamma_widget,E0_widget])
def plotFit(d,F,gamma,E0):
    fig = plt.gcf()
    fig.set_size_inches(15.5, 8.5, forward=True)
    plt.rc('axes', labelsize=18)
    plt.plot(lam*1e9,fit(d,F,gamma,E0)['delta'],'-.',color="xkcd:black",label='Model')
    plt.plot(np.asarray(lam)*1e9,delta_e,color="xkcd:blue",label='Δ')
    plt.plot(lam*1e9,fit(d,F,gamma,E0)['psi'],'-.',color="xkcd:black")
    plt.plot(np.asarray(lam)*1e9,psi_e,color="xkcd:red",label='Ψ')
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("Ψ,Δ (°)")
    plt.legend(prop={'size': 18})
    
    plt.show()
    
    plt.plot(lam*1e9,n(Lorentz(F,gamma,E0,E)),color="xkcd:red")
    plt.plot(lam*1e9,k(Lorentz(F,gamma,E0,E)),color="xkcd:black")
    plt.xlabel("Wavelength (nm)")
    plt.ylabel("n,k")
    plt.show()
    
interactive_plot = interactive(plotFit,d=d_widget,F=F_widget,gamma=gamma_widget,E0=E0_widget)
output = interactive_plot.children[-1]
#output.layout.height = '800px'
button = widgets.Button(description='Auto Fit')
out = widgets.Output()
def on_button_clicked(b):

    func1=lambda tpl : fit(tpl[0],tpl[1],tpl[2],tpl[3])['delta']
    func2=lambda tpl : fit(tpl[0],tpl[1],tpl[2],tpl[3])['psi']
   
    #The function to be minimized
    Error_matrix=lambda tpl,psi_e,delta_e: 1/(2*len(delta_e)-1+1)*(func1(tpl)-delta_e)**2+(func2(tpl)-psi_e)**2 
    #initial parameters
    tplInitial1=(interactive_plot.kwargs['d'],interactive_plot.kwargs['F'],interactive_plot.kwargs['gamma'],interactive_plot.kwargs['E0'])
    # leastsq finds the set of parameters in the tuple tpl that minimizes Error matrix which sum is the MSE
    tplFinal1,success=leastsq(np.sum(Error_matrix),tplInitial1,args=(psi_e,delta_e))
    with out:
        d_widget.value=tplFinal1[0]
        F_widget.value=tplFinal1[1]
        gamma_widget.value=tplFinal1[2]
        E0_widget.value=tplFinal1[3]
        
button.on_click(on_button_clicked)
display(widgets.VBox([button,out]))

interactive_plot

interactive(children=(FloatSlider(value=10.0, description='d', max=50.0, step=1.0), FloatSlider(value=18.0, de…

In [4]:
def Bruggeman(nm,np,f,dep_ab_split,dep_c):
    epsm=nz**2+1j
    epsp=1
    dep_ab_split=0.5
    dep_c=1/3
    f=0
    La=(1-dep_ab_split)*(1-dep_c)
    Lb=dep_ab_split*(1-dep_c)
    Lc=dep_c
    epsa=((f-1+La)*epsm+(La-f)*epsp-np.sqrt(-4*(La-1)*La*epsm*epsp+((1-f-La)*epsm+(f-La)*epsp)**2))/(2*La-2)
    epsb=((f-1+Lb)*epsm+(Lb-f)*epsp-np.sqrt(-4*(Lb-1)*Lb*epsm*epsp+((1-f-Lb)*epsm+(f-Lb)*epsp)**2))/(2*Lb-2)
    epsc=((f-1+Lc)*epsm+(Lc-f)*epsp-np.sqrt(-4*(Lc-1)*Lc*epsm*epsp+((1-f-Lc)*epsm+(f-Lc)*epsp)**2))/(2*Lc-2)
    na=n(epsa)
    ka=k(epsa)
    nb=n(epsb)
    kb=k(epsb)
    nc=n(epsc)
    kc=k(epsc)
    return [na,ka,nb,kb,nc,kc]

#ny=nb
#Beta=3.14/4
#nx=(na*nb)/(np.sqrt(nc**2*np.cos(Beta)**2+na**2*np.sin(Beta)**2))

